In [1]:
import progressbar
import h5py
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import pytz
import warnings
import pylab as plt
%matplotlib inline
from matplotlib.colors import LogNorm
from matplotlib import dates as mdates
import search_dispersion_events 
import importlib
import os
importlib.reload(search_dispersion_events)

<module 'search_dispersion_events' from '/mnt/c/Users/dedasilv/OneDrive - NASA/DMSP/Dispersion/Software/search_dispersion_events.py'>

In [2]:
df_train = pd.read_csv('data/train.csv', parse_dates=['start_time', 'end_time'])
df_val = pd.read_csv('data/val.csv', parse_dates=['start_time', 'end_time'])
df = pd.concat([df_train, df_val]).sort_values('start_time').reset_index()
del df['index']
del df['class']
df.head()

,filename,sat,date,start_time,end_time
0,data/2011/08/05/dms_20110805_17e.001.hdf5,17,2011-08-05,2011-08-05 11:30:00+00:00,2011-08-05 11:33:00+00:00
1,data/2011/08/05/dms_20110805_16e.001.hdf5,16,2011-08-05,2011-08-05 12:55:00+00:00,2011-08-05 12:57:00+00:00
2,data/2011/08/05/dms_20110805_16e.001.hdf5,16,2011-08-05,2011-08-05 12:59:00+00:00,2011-08-05 13:02:00+00:00
3,data/2011/08/05/dms_20110805_18e.001.hdf5,18,2011-08-05,2011-08-05 15:48:00+00:00,2011-08-05 15:51:00+00:00
4,data/2011/08/05/dms_20110805_16e.001.hdf5,16,2011-08-05,2011-08-05 17:03:00+00:00,2011-08-05 17:06:00+00:00


In [5]:
ORBIT_PERIOD = timedelta(minutes=101.6)


def make_plot(index, row, save=False, delta_orbits=[-2, -1, 0, 1, 2], figsize=(27, 5)):
    out_filename = os.path.join('./plots/revisits', f'{index:04d}_' + os.path.basename(row.filename).replace('.hdf5', '.png'))

    if save and os.path.exists(out_filename):
        return
    
    fh = search_dispersion_events.read_file(row.filename)
    fig, axes = plt.subplots(1, len(delta_orbits), figsize=figsize)
    disabled = []
    
    for cnt, delta_orbit in enumerate(delta_orbits):
        delay = delta_orbit * ORBIT_PERIOD # will be negative if delta_orbit < 0
        start_time = row.start_time + delay
        end_time = row.end_time + delay
        
        i = fh['t'].searchsorted(start_time)
        j = fh['t'].searchsorted(end_time)
        
        if i == 0 or j >= fh['t'].size - 1 or max(i, j) - min(i, j) <= 1:
            axes[cnt].set_axis_off()
            disabled.append(cnt)
            #if delta_orbit not in (delta_orbits[0], delta_orbits[-1]):
            #    print(cnt, i, j, fh['t'].shape, np.all(np.diff(fh['t']) > timedelta(seconds=0)))
            continue
            
        im = axes[cnt].pcolor(fh['t'][i:j], np.log10(fh['ch_energy']), fh['ion_d_ener'][:, i:j], 
                              norm=LogNorm(vmin=1e3, vmax=1e8), cmap='jet')
        plt.colorbar(im, ax=axes[cnt]).set_label('Log Energy Flux')
        
        axes[cnt].invert_yaxis()
        axes[cnt].set_ylabel('Ion\nLog Energy [eV] - Ions')

        time_length = end_time - start_time

        title = f'{time_length.total_seconds() / 60:.1f} minutes ; {start_time.isoformat()} to\n{end_time.isoformat()}\n'
        title += 'MLAT = (%.1f deg to %.1f deg)' % (fh['mlat'][i], fh['mlat'][j])
        title += ' Northward' if fh['mlat'][j] > fh['mlat'][i] else ' Southward'
        axes[cnt].set_title(title)

    fig.tight_layout()
    for cnt, ax in enumerate(axes):
        if cnt in disabled:
            continue
        locator = mdates.AutoDateLocator(minticks=3, maxticks=6)
        formatter= mdates.DateFormatter('%H:%M:%S')
        ax.xaxis.set_major_locator(locator)
        ax.xaxis.set_major_formatter(formatter)

    if save:
        try:
            fig.savefig(out_filename)
        except ValueError as e:
            print(e)
        plt.close(fig)

In [6]:
bar = progressbar.ProgressBar()

for index, row in bar(list(df.iterrows())):
    make_plot(index, row, save=True)

 38% (132 of 346) |########              | Elapsed Time: 0:00:14 ETA:   0:06:23<ipython-input-5-4c605d4de762>:54: UserWarning: AutoDateLocator was unable to pick an appropriate interval for this date range. It may be necessary to add an interval value to the AutoDateLocator's intervald dictionary. Defaulting to 30.
  fig.savefig(out_filename)
100% (346 of 346) |######################| Elapsed Time: 0:05:35 Time:  0:05:35
